In [212]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [213]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [214]:
cell_demand = 0.001*164.98*(1.369*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

# D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [215]:
# penalty = 10000 # ton/yr
penalty = 6; # ton/yr

In [216]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [217]:
ipt = []
ipt = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount = ipt

15-element Vector{Float64}:
  49670.291589732
  69512.470736544
  67696.846500888
  69382.78329114
 129687.44540400001
  72376.074329
   5239.896784
   1309.974196
  31111.887154999997
  60258.81301599999
 327493.549
   2258.5762
   1806.86096
 451715.24
 451715.24

In [218]:
D = [0.5, 0.7, 0.8, 1, 1.1, 1.2, 1.4]*cell_demand

7-element Vector{Float64}:
 225857.62
 316200.66799999995
 361372.19200000004
 451715.24
 496886.764
 542058.288
 632401.3359999999

### Solve subproblem to get dual $\pi$ and $\alpha$

In [219]:
function reformu_pi(col_idx, proc_idx, ls_cstr)
    pi_matrix = zeros(ncty, nproc)
    pi = [getdual(con) for con in ls_cstr]
    B = []
    for k in proc_idx
        append!(B, scaler[k] * ones(ncty))
    end
    pi = pi .* B
    pi = reshape(pi, ncty, size(proc_idx)[1])
    pi_matrix[:,col_idx] = sum(pi, dims=2)
    
    return pi_matrix
end

reformu_pi (generic function with 1 method)

In [220]:
function subprob(x_hat, d)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, s >= 0)

    # node output flow constraint
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x_hat[i,k]) for k in 1:nproc for i in 1:ncty]

    # cathode LCA constraints (index=5)
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]

    # cell LCA constraints (index=11)
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]

    # non cell LCA constraints (index=14)
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]

    # battery LCA constraints (index=15)
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    
    @constraint(model, cstr_alp, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + s >= d)
    for j in 1:ncty-1
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
    end 
    
    
    #######################
    tranD = Vector{AffExpr}(undef, ncty)
    tranS= Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
        tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
    end
    
    
    @objective(model, Min, sum(tranD-tranS) + s*penalty)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_s = JuMP.value.(s)
    
    
    ####################### 
    op_pi = [getdual(con) for con in cstr_op]
    op_pi = reshape(op_pi, ncty, nproc)

    cth_pi = reformu_pi(5, cathode, cstr_cth)
    cell_pi = reformu_pi(11, cell, cstr_cell)
    noncell_pi = reformu_pi(14, noncell, cstr_noncell)
    battery_pi = reformu_pi(15, battery, cstr_battery)
    pi_matrix = op_pi + cth_pi + cell_pi + noncell_pi + battery_pi
    
    alp = getdual(cstr_alp)

    
    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "s_opt"=>sub_s, "pi"=>pi_matrix, "alp"=>alp]);
    return result
    
end


subprob (generic function with 1 method)

### Solve master problem

In [221]:
M = -5e7

-5.0e7

In [222]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
        end
    end
    for k in 1:nproc
        @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    proM = x * price

    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, sum(x .* G) + theta >= g)
        end
    end
    
    @objective(model, Min, sum(proD-proS)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result
end

masterprob (generic function with 1 method)

### Generate cuts

In [223]:
function add_cuts(x, D)
    G = [] # gradient
    g = 0 # intersection
    qy = 0
    
    for d in D
        res = subprob(x, d)
        gradient = -1 * res["pi"]
        
        push!(G, gradient)
        g += d * res["alp"]
        qy += res["qyhat"]
    end
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    z_hat = sum(proD - proS) + qy/size(D)[1]
    Exp_G = sum(G) / size(D)[1]
    Exp_g = sum(g) / size(D)[1]
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat
end


add_cuts (generic function with 1 method)

### L Shape Method

In [255]:
function main(toler)
    i = 0
    z_ub = 1e7
    x_opt = 0
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat, D)
        if z_hat < z_ub
            z_ub = z_hat
            x_opt = x_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
#     println(z_lb, z_ub, x_opt, i)
    
end

main (generic function with 1 method)

In [256]:
result = main(0.005)

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only -

Dict{String, Any} with 4 entries:
  "z_lb"      => -7.93916e6
  "x_opt"     => [6200.0 0.0 … 0.0 0.0; 1770.29 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.…
  "iteration" => 13
  "z_ub"      => -7.91652e6

In [259]:
xopt = result["x_opt"]

30×15 Matrix{Float64}:
  6200.0       0.0      0.0      0.0   …     0.0   0.0        0.0
  1770.29      0.0      0.0      0.0         0.0   0.0        0.0
  2200.0       0.0  67696.8  67000.0         0.0   0.0        0.0
   500.0    3100.0      0.0   2382.78        0.0   0.0        0.0
 39000.0       0.0      0.0      0.0         0.0   4.51715e5  0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0   66412.5      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0      0.0      0.0         0.0   0.0  

In [260]:
res_x = DataFrame(xopt, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries)

Row,country,Li,Co,Mn,Ni,cathode,graphite,PP,PE,Cu,Al,cell,PET,electronics,noncell,battery
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,6200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia,1770.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brazil,2200.0,0.0,67696.8,67000.0,0.0,62376.1,5239.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Canada,500.0,3100.0,0.0,2382.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Chile,39000.0,0.0,0.0,0.0,0.0,0.0,0.0,1309.97,31111.9,0.0,0.0,2258.58,0.0,4.51715e5,0.0
6,China,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Colombia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Congo,0.0,66412.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [261]:
CSV.write("/Users/bourg/Desktop/Oct31/eco_stoc.csv", res_x) 

"/Users/bourg/Desktop/Oct31/eco_stoc.csv"

In [250]:
# i = 0
# z_ub = 1e7
# x_opt = 0

# cuts = []
# res0 = masterprob(cuts)
# x_hat = res0["x_hat"]
# z_lb = res0["z_lb"]

# for i in 1:15
#     new_cut, z_hat = add_cuts(x_hat, D)
#     if z_hat < z_ub
#         z_ub = z_hat
#         x_opt = x_hat
#     end

#     push!(cuts, new_cut)

#     res1 = masterprob(cuts)
#     z_lb = res1["z_lb"]
#     x_hat = res1["x_hat"]

# end

# println(z_lb, z_ub)


In [245]:
z_lb = -7.932124752515396e6
z_ub = -7.91652262914418e6

-7.91652262914418e6

In [247]:
(z_ub - z_lb) <= 0.01 * min(abs(z_ub), abs(z_lb))

true

In [227]:
zlb = -8.01779695932381e6                
zub = -7.91652262914418e6;

In [242]:
(zub - z_lb) > 0.02 * min(abs(zub), abs(zlb))

false

791652.262914418

---

In [54]:
i = 0
z_ub = 1e7
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2023-11-27


-1.0838412704242925e8

In [55]:
new_cut, z_hat = add_cuts(x_hat, D)

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27


(Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.2113261655335346 4.368008737708988; -0.16449709458296913 -0.0 … 11.774861570087412 9.483831396993969; … ; 2.6613651079346035 1.8976178674364448 … 14.600020319491604 12.174982796330704; -0.0 -0.0 … 5.658420887375434 3.1612097799728622], "intersection" => 1.8197671097142857e6), -7.91652262914418e6)

In [56]:
new_cut

Dict{String, Any} with 2 entries:
  "gradient"     => [-0.127812 0.0898899 … 2.21133 4.36801; -0.164497 -0.0 … 11…
  "intersection" => 1.81977e6

In [57]:
push!(cuts, new_cut)

1-element Vector{Any}:
 Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.2113261655335346 4.368008737708988; -0.16449709458296913 -0.0 … 11.774861570087412 9.483831396993969; … ; 2.6613651079346035 1.8976178674364448 … 14.600020319491604 12.174982796330704; -0.0 -0.0 … 5.658420887375434 3.1612097799728622], "intersection" => 1.8197671097142857e6)

In [125]:
model = Model(Gurobi.Optimizer)
@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, theta >= M)

for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end
for k in 1:nproc
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
proM = x * price

pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = (x.*pro_sink)*ones(nproc,1);

Academic license - for non-commercial use only - expires 2023-11-27


In [126]:
G = new_cut["gradient"]
g = new_cut["intersection"]

1.8197671097142857e6

In [127]:
@constraint(model, sum(x .* G) + theta >= g);

In [128]:
@objective(model, Min, sum(proD-proS)+theta)
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 466 rows, 451 columns and 1294 nonzeros
Model fingerprint: 0xc3f55207
Coefficient statistics:
  Matrix range     [2e-02, 2e+01]
  Objective range  [7e-03, 3e+02]
  Bounds range     [1e+08, 1e+08]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 460 rows and 399 columns
Presolve time: 0.00s
Presolved: 6 rows, 52 columns, 93 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -1.0838413e+08   3.139613e+06   0.000000e+00      0s
       5   -1.4442859e+07   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds
Optimal objective -1.444285937e+07

User-callback calls 43, time in user-callback 0.00 sec


In [129]:
x_hat2 = JuMP.value.(x)

30×15 Matrix{Float64}:
  6200.0       0.0      0.0      0.0   …     0.0   0.0        0.0
  1370.29      0.0      0.0      0.0         0.0   0.0        0.0
  2200.0       0.0  57596.8  67000.0         0.0   0.0        0.0
   500.0    3100.0      0.0   2382.78        0.0   0.0        0.0
 39000.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0   66412.5      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0      0.0      0.0         0.0   0.0  

In [123]:
x_hat1 = JuMP.value.(x)

30×15 Matrix{Float64}:
  6200.0       0.0      0.0      0.0   …     0.0   0.0        0.0
  1770.29      0.0      0.0      0.0         0.0   0.0        0.0
  2200.0       0.0  67696.8  67000.0         0.0   0.0        0.0
   500.0    3100.0      0.0   2382.78        0.0   0.0        0.0
 39000.0       0.0      0.0      0.0         0.0   4.51715e5  0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0   66412.5      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0   …     0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     0.0       0.0      0.0      0.0         0.0   0.0        0.0
     ⋮                                 ⋱                      
     0.0       0.0      0.0      0.0         0.0   0.0  

In [130]:
theta_hat2 = JuMP.value.(theta)

-7.073771924593389e6

In [94]:
z_lb = JuMP.objective_value(model)

-1.44428593698294e7

In [100]:
i = 0
z_ub = 1e7
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

new_cut, z_hat = add_cuts(x_hat, D)

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27


(Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.2113261655335346 4.368008737708988; -0.16449709458296913 -0.0 … 11.774861570087412 9.483831396993969; … ; 2.6613651079346035 1.8976178674364448 … 14.600020319491604 12.174982796330704; -0.0 -0.0 … 5.658420887375434 3.1612097799728622], "intersection" => 1.8197671097142857e6), -7.91652262914418e6)

In [102]:
z_hat, z_ub

(-7.91652262914418e6, 1.0e7)

In [104]:
z_ub = z_hat
x_opt = x_hat;

In [106]:
push!(cuts, new_cut)

1-element Vector{Any}:
 Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.2113261655335346 4.368008737708988; -0.16449709458296913 -0.0 … 11.774861570087412 9.483831396993969; … ; 2.6613651079346035 1.8976178674364448 … 14.600020319491604 12.174982796330704; -0.0 -0.0 … 5.658420887375434 3.1612097799728622], "intersection" => 1.8197671097142857e6)

In [107]:
res1 = masterprob(cuts)

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 2 entries:
  "x_hat" => [6200.0 0.0 … 0.0 0.0; 1370.29 0.0 … 0.0 0.0; … ; 0.0 0.0 … 4.5171…
  "z_lb"  => -1.44429e7

In [111]:
z_lb = res1["z_lb"]

-1.44428593698294e7

In [113]:
x_hat = res1["x_hat"];

In [114]:
add_cuts(x_hat, D)

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27


(Dict{String, Any}("gradient" => [-0.12781186121027854 0.08988994647713167 … 2.3995522894787125 4.620436383490762; -0.16449709458296913 -0.0 … 0.8340864939957929 9.431066318012025; … ; 1.0590860694264475 1.8976178674364448 … -0.00993317862443092 12.174982796330704; -0.0 -0.0 … 0.919324128563967 2.8158268387991185], "intersection" => 1.8197671097142857e6), -6.646553849579561e6)

In [115]:
subprob(x_hat, D[1])

Academic license - for non-commercial use only - expires 2023-11-27


Dict{String, Any} with 5 entries:
  "y_opt" => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 …
  "s_opt" => 0.0
  "pi"    => [0.127812 -0.0898899 … -2.39955 -2.63577; 0.164497 0.0 … -0.834086…
  "alp"   => 0.0
  "qyhat" => 4.51504e5

In [172]:
x_hat = x_hat1;

In [173]:
d = D[1]

225857.62

In [174]:
for d in D
    model = Model(Gurobi.Optimizer)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, s >= 0)

    # node output flow constraint
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x_hat[i,k]) for k in 1:nproc for i in 1:ncty]

    # cathode LCA constraints (index=5)
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]

    # cell LCA constraints (index=11)
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]

    # non cell LCA constraints (index=14)
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]

    # battery LCA constraints (index=15)
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, cstr_alp, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + s >= d)
    for j in 1:ncty-1
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0)
    end 


    #######################
    tranD = Vector{AffExpr}(undef, ncty)
    tranS= Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += (amount * distance[!, 2:end][i,j] * EF_trans)
        end
        tranD[j] = arc_emi
        tranS[j] = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
    end


    @objective(model, Min, sum(tranD-tranS) + s*penalty)
    JuMP.optimize!(model)
end

Academic license - for non-commercial use only - expires 2023-11-27
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 900 rows, 13501 columns and 27001 nonzeros
Model fingerprint: 0x0c7eff29
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 5e+05]
Presolve removed 900 rows and 13501 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9657527e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.965752693e+05

User-callback calls 29, time in user-callback 0.00 sec


In [175]:
qy_hat1 = JuMP.objective_value(model)
sub_y1 = JuMP.value.(y)
sub_s1 = JuMP.value.(s)

0.0

In [176]:
op_pi1 = [getdual(con) for con in cstr_op]
op_pi1 = reshape(op_pi1, ncty, nproc)

cth_pi1 = reformu_pi(5, cathode, cstr_cth)
cell_pi1 = reformu_pi(11, cell, cstr_cell)
noncell_pi1 = reformu_pi(14, noncell, cstr_noncell)
battery_pi1 = reformu_pi(15, battery, cstr_battery)
pi_matrix1 = op_pi1 + cth_pi1 + cell_pi1 + noncell_pi1 + battery_pi1

alp1 = getdual(cstr_alp)

0.0

In [156]:
qy_hat2 = JuMP.objective_value(model)
sub_y2 = JuMP.value.(y)
sub_s2 = JuMP.value.(s)

0.0

In [157]:
op_pi2 = [getdual(con) for con in cstr_op]
op_pi2 = reshape(op_pi2, ncty, nproc)

cth_pi2 = reformu_pi(5, cathode, cstr_cth)
cell_pi2 = reformu_pi(11, cell, cstr_cell)
noncell_pi2 = reformu_pi(14, noncell, cstr_noncell)
battery_pi2 = reformu_pi(15, battery, cstr_battery)
pi_matrix2 = op_pi2 + cth_pi2 + cell_pi2 + noncell_pi2 + battery_pi2

alp2 = getdual(cstr_alp)

30×15 Matrix{Float64}:
  0.127812   -0.0898899  -2.02277     …  -2.15795   -2.39955      -2.63577
  0.164497    0.0        -0.583463       -6.11574   -0.834086     -6.19768
  0.062538    0.0         0.062538       -3.35342   -0.32557      -5.87118
  0.128633    0.128633   -1.81422        -1.93942   -2.19593      -5.3938
  0.111558    0.0        -1.67871        -1.64678   -2.07691     -18.9181
  0.0         0.0        -0.578394    …  -1.37385   -0.953965     -1.37385
 -0.0223146  -0.240016   -2.13377        -2.30807   -2.51391      -4.03107
 -0.152984    0.13146    -0.00776108     -6.91252   -0.214301     -8.88406
  0.0         0.0        -1.5092         -1.8562    -1.8891       -3.59509
  0.0         0.0        -0.609721       -0.493715  -0.985292     -0.493715
  0.0         0.0        -1.22293     …  -1.59942   -1.5985       -1.59942
  0.0         0.0        -0.0584193      -2.48233   -0.433991     -2.48233
  0.0         0.0        -0.371318       -0.860489  -0.74689      -0.860489
  

In [160]:
op_pi1 == op_pi2

false

In [161]:
cth_pi1 == cth_pi2

false

In [164]:
cell_pi1 == cell_pi2

false

In [165]:
noncell_pi1 == noncell_pi2

false

In [166]:
battery_pi1 == battery_pi2

false

In [169]:
pi_matrix1[:,1] == pi_matrix2[:,2]

false

In [171]:
for i in 1:30
    if pi_matrix1[i,1] != pi_matrix2[i,1]
        println((pi_matrix1[i,1], pi_matrix2[i,1]))
    end
end

(-0.6322869704056888, -0.1529843099906445)
(-0.1135170989960177, 0.0)
(-1.461923867459026, 0.1403551710491303)
(-1.1897393305285533, 0.0)
(-0.08761891679301259, 0.0)
(-0.868654925031171, 0.0)
(-1.0123247094337913, 0.0)
(-2.6613651079346035, -1.0590860694264475)


In [182]:
x = x_hat2;

In [183]:
G = []
g = []
qy = []
for d in D
    res = subprob(x, d)
    gradient = -1 * res["pi"]
    itc = d * res["alp"]

    push!(G, gradient)
    push!(g, itc)
    push!(qy, res["qyhat"])
end

Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27
Academic license - for non-commercial use only - expires 2023-11-27


In [181]:
G1 = G
g1 = g
qy1 = qy

7-element Vector{Any}:
 196575.26928507572
 196575.26928507572
 196575.26928507572
 196575.26928507572
 467604.4132850759
 738633.5572850754
      1.2806918452850753e6

In [184]:
G2 = G
g2 = g
qy2 = qy

7-element Vector{Any}:
 451504.45165644935
 451504.45165644935
 451504.45165644935
 451504.45165644935
 722533.5956564496
 993562.7396564491
      1.5356210276564488e6

In [198]:
G11 = sum(G1) / 7

30×15 Matrix{Float64}:
 -0.127812    0.0898899   3.31216   …   2.15795    2.21133     4.36801
 -0.164497   -0.0         2.76232       6.11574   11.7749      9.48383
 -0.062538   -0.0        -0.0           3.35342    9.02438     9.02996
 -0.128633   -0.128633    3.09363       1.93942    0.967228    5.21857
 -0.111558   -0.0         2.80099       1.64678    0.0226337   8.08391
 -0.0        -0.0         2.23935   …   1.37385    7.03227     4.60723
  0.0223146   0.240016    3.46228       2.30807    2.33557     5.05207
  0.632287   -0.13146     3.5591        6.91252   12.5891     13.6793
 -0.0        -0.0         3.01041       1.8562     2.44871     4.88062
 -0.0        -0.0         1.09355       0.493715   6.15214     3.7271
 -0.0        -0.0         2.75363   …   1.59942    3.68458     3.97549
 -0.0        -0.0         0.745077      2.48233    8.14075     5.71571
 -0.0        -0.0         0.929458      0.860489   6.51891     4.09387
  ⋮                                 ⋱                   

In [199]:
G22 = sum(G2) / 7

30×15 Matrix{Float64}:
 -0.127812    0.0898899   2.02277     …   2.15795    2.39955      4.62044
 -0.164497   -0.0         0.583463        6.11574    0.834086     9.43107
 -0.062538   -0.0        -0.062538        3.35342    0.32557      9.10456
 -0.128633   -0.128633    1.81422         1.93942    2.19593      7.54189
 -0.111558   -0.0         1.67871         1.64678    2.07691     21.2529
 -0.0        -0.0         0.578394    …   1.37385    0.953965     4.60723
  0.0223146   0.240016    2.13377         2.30807    2.51391      5.92503
  0.152984   -0.13146     0.00776108      6.91252    0.214301    12.1174
 -0.0        -0.0         1.5092          1.8562     1.8891       5.80706
 -0.0        -0.0         0.609721        0.493715   0.985292     3.7271
 -0.0        -0.0         1.22293     …   1.59942    1.5985       3.97549
 -0.0        -0.0         0.0584193       2.48233    0.433991     5.71571
 -0.0        -0.0         0.371318        0.860489   0.74689      4.09387
  ⋮               

In [208]:
g11 = sum(g1)/7

1.8197671097142857e6

In [209]:
g22 = sum(g2)/7

1.8197671097142857e6

In [210]:
g1 == g2

true